In [1]:
import torch

class CustomReLU(torch.nn.Module):
    """
    Custom implementation of the ReLU activation function.
    """
    def __init__(self):
        super(CustomReLU, self).__init__()

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Applies the ReLU function element-wise: max(0, x).
        Args:
            x (torch.Tensor): Input tensor
        Returns:
            torch.Tensor: Output tensor with ReALU applied
        """
        return torch.maximum(x, torch.zeros_like(x))

In [2]:
import torch

class ConvolutionalLayer:
    """
    A utility class to create a convolutional layer with a custom ReLU activation.
    """
    @staticmethod
    def create(in_channels: int, out_channels: int, kernel_size: int, stride: int, padding: int) -> torch.nn.Sequential:
        """
        Creates a convolutional layer with a custom ReLU activation.
        Args:
            in_channels (int): Number of input channels
            out_channels (int): Number of output channels
            kernel_size (int): Size of the convolution kernel
            stride (int): Stride of the convolution
            padding (int): Padding added to the input
        Returns:
            torch.nn.Sequential: A sequential layer containing Conv2d and CustomReLU(+Batch Normalization)
        """
        return torch.nn.Sequential(
            torch.nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=padding),
            torch.nn.BatchNorm2d(out_channels),
            CustomReLU()
        )

In [3]:
import torch

class FullyConnectedLayer:
    """
    A utility class to create Fully Connected (FC) layers with Xavier initialization.
    """
    @staticmethod
    def Dense(input_dim: int, output_dim: int) -> torch.nn.Linear:
        """
        Creates an FC layer and applies Xavier initialization.
        Args:
            input_dim (int): Number of input features
            output_dim (int): Number of output features
        Returns:
            torch.nn.Linear: Initialized Fully-Connected layer
        """
        layer = torch.nn.Linear(input_dim, output_dim, bias=True)
        torch.nn.init.xavier_uniform_(layer.weight)
        return layer

In [4]:
import torch

class Inception_A(torch.nn.Module):
    """
    Implementation of the Inception A layer(Base on Figure 5) introduced by GoogleNet_V2 and V3

    Branch 1: 1x1 Convolution
    Branch 2: 1×1 → 3×3 Convolution
    Branch 3: 1×1 → 3×3 → 3×3 Convolution
    Branch 4: Average Pooling → 1×1 Convolution
    """
    def __init__(self, in_channels: int, out_branch1: int, reduce_branch2: int, out_branch2: int, 
                 reduce_branch3: int, out_branch3: int, out_branch4: int) -> torch.nn.Sequential:
        """
        Creates a Inception layer with a custom ReLU activation Function.
        Args:
            in_channels (int): Number of input channels
            out_branch1 (int): Number of output channels for Branch 1
            reduce_branch2 (int): Number of output channels for 3x3 convolution reduction
            out_branch2 (int): Number of output channels for Branch 2
            reduce_branch3 (int): Number of output channels for 3x3 convolution reduction
            out_branch3 (int): Number of output channels for Branch 3
            out_branch4 (int): Number of output channels for Branch 4
        Returns:
            torch.nn.Sequential: A sequential block representing the Inception_A layer
        """
        super(Inception_A, self).__init__()

        self.branch1 = ConvolutionalLayer.create(in_channels, out_branch1, 1, 1, 0)
        
        self.branch2 = torch.nn.Sequential(
            ConvolutionalLayer.create(in_channels, reduce_branch2, 3, 1, 1),
            ConvolutionalLayer.create(reduce_branch2, out_branch2, 3, 1, 1)
        )

        self.branch3 = torch.nn.Sequential(
            ConvolutionalLayer.create(in_channels, reduce_branch3, 3, 1, 1),
            ConvolutionalLayer.create(reduce_branch3, out_branch3, 3, 1, 1),
            ConvolutionalLayer.create(out_branch3, out_branch3, 3, 1, 1)
        )

        self.branch4 = torch.nn.Sequential(
            torch.nn.AvgPool2d(kernel_size=3, stride=1, padding=1),
            ConvolutionalLayer.create(in_channels, out_branch4, 1, 1, 0)
        )
        
    def forward(self, x):
        """
        Defines the forward pass of the InceptionV1 layer.
        Args:
            x (torch.Tensor): Input tensor of shape (batch_size, in_channels, height, width)

        Returns:
            torch.Tensor: Output tensor after passing through all branches of the Inception_A layer
        """
        return torch.cat([self.branch1(x), self.branch2(x), self.branch3(x), self.branch4(x)], dim=1)

In [5]:
import torch

class Inception_B(torch.nn.Module):
    """
    Implementation of the Inception B layer(Base on Figure 6) introduced by GoogleNet_V2 and V3

    Branch 1: 1x1 Convolution
    Branch 2: 1×1 → 1×7 → 7×1 Convolution
    Branch 3: 1×1 → 7×1 → 1×7 → 7×1 → 1×7 Convolution
    Branch 4: Max Pooling → 1×1 Convolution
    """
    def __init__(self, in_channels: int, out_channels: int) -> torch.nn.Sequential:
        """
        Creates a Inception layer with a custom ReLU activation Function.
        Args:
            in_channels (int): Number of input channels
            out_channels (int): Number of output channels
        Returns:
            torch.nn.Sequential: A sequential block representing the Inception_B layer
        """
        super(Inception_B, self).__init__()

        self.branch1 = ConvolutionalLayer.create(in_channels, 192, 1, 1, 0)  # Output value is fixed to 192
        
        self.branch2 = torch.nn.Sequential(
            ConvolutionalLayer.create(in_channels, 192, 1, 1, 0),
            ConvolutionalLayer.create(192, 384, (1, 7), 1, (0, 3)),
            ConvolutionalLayer.create(384, 384, (7, 1), 1, (3, 0))
        )

        self.branch3 = torch.nn.Sequential(
            ConvolutionalLayer.create(in_channels, 192, 1, 1, 0),
            ConvolutionalLayer.create(192, 384, (1, 7), 1, (0, 3)),
            ConvolutionalLayer.create(384, 384, (7, 1), 1, (3, 0)),
            ConvolutionalLayer.create(384, 384, (1, 7), 1, (0, 3)),
            ConvolutionalLayer.create(384, 384, (7, 1), 1, (3, 0))
        )

        self.branch4 = torch.nn.Sequential(
            torch.nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
            ConvolutionalLayer.create(in_channels, 320, 1, 1, 0)    # Output value is fixed to 320
        )
        
    def forward(self, x):
        """
        Defines the forward pass of the InceptionV1 layer.
        Args:
            x (torch.Tensor): Input tensor of shape (batch_size, in_channels, height, width)

        Returns:
            torch.Tensor: Output tensor after passing through all branches of the Inception_A layer
        """
        return torch.cat([self.branch1(x), self.branch2(x), self.branch3(x), self.branch4(x)], dim=1)

In [6]:
import torch

class Inception_C(torch.nn.Module):
    """
    Implementation of the Inception C layer(Base on Figure 7) introduced by GoogleNet_V2 and V3

    Branch 1: 1x1 Convolution
    Branch 2: 1×1 → (1×3 + 3×1) Convolution
    Branch 3: 1×1 → 3×3 → (1×3 + 3×1) Convolution
    Branch 4: Average Pooling → 1×1 Convolution
    """
    def __init__(self, in_channels: int, out_branch1: int, reduce_branch2: int, 
                 reduce_branch3: int, out_branch4: int) -> torch.nn.Sequential:
        """
        Creates a Inception layer with a custom ReLU activation Function.
        Args:
            in_channels (int): Number of input channels
            out_branch1 (int): Number of output channels for Branch 1
            reduce_branch2 (int): Number of output channels for 3x3 convolution reduction
            reduce_branch3 (int): Number of output channels for 3x3 convolution reduction
            out_branch4 (int): Number of output channels for Branch 4
        Returns:
            torch.nn.Sequential: A sequential block representing the Inception_A layer
        """
        super(Inception_C, self).__init__()

        self.branch1 = ConvolutionalLayer.create(in_channels, out_branch1, 1, 1, 0)
        
        self.branch2_a = ConvolutionalLayer.create(in_channels, 384, 1, 1, 0)
        self.branch2_b1 = ConvolutionalLayer.create(384, reduce_branch2, (1, 3), 1, (0, 1))
        self.branch2_b2 = ConvolutionalLayer.create(384, reduce_branch2, (3, 1), 1, (1, 0))

        self.branch3_a = ConvolutionalLayer.create(in_channels, 384, 1, 1, 0)
        self.branch3_b = ConvolutionalLayer.create(384, 448, 1, 1, 0)
        self.branch2_c1 = ConvolutionalLayer.create(448, reduce_branch3, (1, 3), 1, (0, 1))
        self.branch2_c2 = ConvolutionalLayer.create(448, reduce_branch3, (3, 1), 1, (1, 0))

        self.branch4 = torch.nn.Sequential(
            torch.nn.AvgPool2d(kernel_size=3, stride=1, padding=1),
            ConvolutionalLayer.create(in_channels, out_branch4, 1, 1, 0)
        )
        
    def forward(self, x):
        """
        Defines the forward pass of the InceptionV1 layer.
        Args:
            x (torch.Tensor): Input tensor of shape (batch_size, in_channels, height, width)

        Returns:
            torch.Tensor: Output tensor after passing through all branches of the Inception_A layer
        """
        x_b1 = self.branch1(x)

        x_b2_a = self.branch2_a(x)
        x_b2_b1 = self.branch2_b1(x_b2_a)
        x_b2_b2 = self.branch2_b2(x_b2_a)
        x_b2_cat = torch.cat([x_b2_b1, x_b2_b2], dim=1)

        x_b3_a = self.branch3_a(x)
        x_b3_b = self.branch3_b(x_b3_a)
        x_b3_c1 = self.branch3_c1(x_b3_b)
        x_b3_c2 = self.branch3_c2(x_b3_b)
        x_b3_cat = torch.cat([x_b3_c1, x_b3_c2], dim=1)

        x_b4 = self.branch4(x)

        return torch.cat([x_b1, x_b2_cat, x_b3_cat, x_b4], dim=1)

In [7]:
import torch

class CustomGoogLeNet_V3(torch.nn.Module):
    """
    Implementation of the GoogLeNet_V2 and V3 model.
    Input: Image tensor (batch_size, 3, 299, 299)
    Output: Class scores (batch_size, 1000)
    """
    def __init__(self, dropout_rate=0.4):
        """
        Args:
            dropout_rate (float): Dropout Rate = 0.4(Base on Paper)
        """
        super(CustomGoogLeNet_V3, self).__init__()

        # Convolutional and pooling layers
        self.layer1 = torch.nn.Sequential(
            ConvolutionalLayer.create(3, 32, 3, 2, 0),
            ConvolutionalLayer.create(32, 32, 3, 1, 0),
            ConvolutionalLayer.create(32, 64, 3, 1, 1),
            torch.nn.MaxPool2d(kernel_size=3, stride=2, padding=0)
        )

        self.layer2 = torch.nn.Sequential(
            ConvolutionalLayer.create(64, 80, 3, 1, 0),
            ConvolutionalLayer.create(80, 192, 3, 2, 0),
            ConvolutionalLayer.create(192, 288, 3, 1, 1),
            torch.nn.MaxPool2d(kernel_size=3, stride=1, padding=1)
        )

        self.layer3 = torch.nn.Sequential(
            Inception_A(192, 64, 48, 64, 64, 96, 32),
            Inception_A(256, 64, 48, 64, 64, 96, 64),
            Inception_A(288, 128, 128, 256, 128, 256, 128),
            torch.nn.MaxPool2d(kernel_size=3, stride=2, padding=0)
        )

        self.layer4 = torch.nn.Sequential(
            Inception_B(768, 1280),
            Inception_B(1280, 1280),
            Inception_B(1280, 1280),
            Inception_B(1280, 1280),
            Inception_B(1280, 1280),
            torch.nn.MaxPool2d(kernel_size=3, stride=2, padding=0)
        )

        self.layer5 = torch.nn.Sequential(
            Inception_C(1280, 256, 192, 192, 128),
            Inception_C(1152, 384, 352, 352, 256),
            torch.nn.AvgPool2d(kernel_size=8, stride=1, padding=0)
        )

        # Fully Connected layers and dropout
        self.layer_drop = torch.nn.Dropout(p=dropout_rate)

        self.layer6 = FullyConnectedLayer.Dense(1 * 1 * 2048, 1000)

        self.layer7 = FullyConnectedLayer.Dense(1000, 1000)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Defines the forward pass of the model.
        Args:
            x (torch.Tensor): Input image tensor (batch_size, 3, 224, 224)
        Returns:
            torch.Tensor: Class scores (batch_size, 1000)
        """
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
        x = self.layer_drop(x)
        x = x.view(x.size(0), -1)  # Flatten
        x = self.layer6(x)
        x = self.layer7(x)
        return x

In [8]:
from torchsummary import summary

model = CustomGoogLeNet_V3()
summary(model, input_size=(3, 299, 299))

RuntimeError: Given groups=1, weight of size [64, 192, 1, 1], expected input[2, 288, 35, 35] to have 192 channels, but got 288 channels instead